In [96]:
import pandas as pd 
import pip._vendor.requests as requests
import spotipy
import numpy as np
import matplotlib as plt
from spotipy.oauth2 import SpotifyOAuth
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 
from  sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
import pickle
scope1 = "user-library-read"
from spotipy.oauth2 import SpotifyClientCredentials
sp =spotipy.Spotify(auth_manager= SpotifyOAuth(client_id = "15fd015d4da742d0bf7da9c6ed3120c7", client_secret = "349c5181636e4e3cbb974b051db425fd", redirect_uri = "http://localhost", scope= scope1))

Happy song analysis

In [97]:
# df_happy1 = pd.read_csv("happy_songs_everyone_knows__.csv")
# df_happy2 = pd.read_csv("jhjh.csv")
# df_happy_total = pd.concat([df_happy1, df_happy2])
# df_happy_total = df_happy_total.drop(["Spotify ID", "Artist IDs", "Track Name", "Album Name", "Artist Name(s)", "Release Date"], axis = 1)
# df_happy_total["tag"] = "happy"
# df_happy_total.describe()
# # df_happy1[["Danceability","Energy","Speechiness","Acousticness", "Instrumentalness", "Liveness","Valence"]].plot.box(figsize = (10, 10))
# # # a = df_happy1.plot.scatter(x = "Popularity", y= "Energy")
# # # a
# # boxplot = df_happy1.boxplot(column =["Popularity","Danceability","Energy","Speechiness","Acousticness","Instrumentalness", "Liveness"])
# # boxplot
# # plt.show()

In [118]:
df_total = pd.read_csv("total.csv")

In [99]:
# df_total = df_total[~df_total['Tempo'].isin(df_vibe['Tempo'])]

In [100]:
df_total.loc[df_total["tag"] == "calm", "tag"] = 0
df_total.loc[df_total["tag"] == "happy", "tag"] = 1
df_total.loc[df_total["tag"] == "energ", "tag"] = 2
df_total.loc[df_total["tag"] == "sad", "tag"] = 3

In [101]:
df_total_after = df_total
df_total

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,tag
0,0.746,0.754,1,-6.684,0.0676,0.01520,0.000000,0.0486,0.746,149.882,2
1,0.628,0.593,8,-5.036,0.0323,0.09300,0.000002,0.0960,0.104,125.026,2
2,0.620,0.805,5,-5.538,0.0417,0.00564,0.000092,0.2850,0.586,122.062,2
3,0.504,0.785,8,-4.802,0.1040,0.07380,0.000000,0.2390,0.502,173.968,2
4,0.560,0.872,3,-4.269,0.1400,0.02390,0.000007,0.3710,0.682,126.050,2
...,...,...,...,...,...,...,...,...,...,...,...
1496,0.688,0.662,6,-5.570,0.0441,0.09710,0.000001,0.1720,0.549,115.106,1
1497,0.521,0.877,7,-5.444,0.0683,0.01320,0.080600,0.0929,0.199,180.007,1
1498,0.287,0.473,5,-3.551,0.0340,0.08580,0.000044,0.3830,0.577,186.283,1
1499,0.673,0.704,6,-8.056,0.3600,0.19600,0.000000,0.0898,0.372,86.919,1


In [102]:

# Normalize the DataFrame to be between 0 and 1
min_value = df_total[["Tempo"]].min()
max_value = df_total[["Tempo"]].max()
df_total["Tempo"] = (df_total[["Tempo"]] - min_value) / (max_value - min_value)

min_value = df_total[["Loudness"]].min()
max_value = df_total[["Loudness"]].max()
df_total["Loudness"] = (df_total[["Loudness"]] - min_value) / (max_value - min_value)

min_value = df_total[["Key"]].min()
max_value = df_total[["Key"]].max()
df_total["Key"] = (df_total[["Key"]] - min_value) / (max_value - min_value)



In [133]:
# Normalize the DataFrame to be between 0 and 1 function 
def clean_and_normalize(inpput):
    inpput = inpput[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]]
    min_value = inpput[["Tempo"]].min()
    max_value = inpput[["Tempo"]].max()
    inpput["Tempo"] = (inpput[["Tempo"]] - min_value) / (max_value - min_value)

    min_value = inpput[["Loudness"]].min()
    max_value = inpput[["Loudness"]].max()
    inpput["Loudness"] = (inpput[["Loudness"]] - min_value) / (max_value - min_value)

    min_value = inpput[["Key"]].min()
    max_value = inpput[["Key"]].max()
    inpput["Key"] = (inpput[["Key"]] - min_value) / (max_value - min_value)
    inpput



In [103]:
[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]]data = df_total

In [104]:
data_target = df_total[["tag"]].astype(int)
data_target.dtypes

tag    int64
dtype: object

In [105]:
# song_x_train, song_x_test, song_y_train, song_y_test = train_test_split(data, data_target, test_size= 0.2, random_state= 42)
# df_result = []

# for i in range(10):
#     glp = GradientBoostingClassifier(n_estimators=170, learning_rate=0.001, max_depth=5, min_samples_split=2, min_samples_leaf=1, random_state=42)
#     song_y_train = np.ravel(song_y_train)
#     song_y_test = np.ravel(song_y_test)
#     glp.fit(song_x_train,song_y_train)
#     song_y_predict = glp.predict(song_x_test)
#     output = {"RMSE": np.sqrt(mean_squared_error(song_y_test,song_y_predict)), "accuracy_score":metrics.accuracy_score(song_y_test, song_y_predict)}
#     df_result.append(output)
#         # print("RMSE", np.sqrt(mean_squared_error(song_y_test,song_y_predict)))
#         # print(metrics.accuracy_score(song_y_test, song_y_predict))
#         # print("------")
# df_output = pd.DataFrame(df_result)
# df_output

In [140]:
song_x_test

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
1117,0.766,0.2830,0.727273,0.775361,0.0846,0.1110,0.017200,0.1150,0.144,0.326002
643,0.330,0.0109,0.545455,0.303344,0.0356,0.9820,0.924000,0.1120,0.208,0.237233
422,0.366,0.0329,0.909091,0.380450,0.0329,0.9940,0.909000,0.1160,0.211,0.340815
413,0.610,0.5080,0.727273,0.861144,0.1520,0.2970,0.000000,0.3840,0.758,0.822876
451,0.536,0.0409,0.000000,0.294008,0.0542,0.9920,0.946000,0.1070,0.282,0.182605
...,...,...,...,...,...,...,...,...,...,...
610,0.290,0.0197,0.454545,0.443329,0.0443,0.9940,0.903000,0.1110,0.319,0.090118
1343,0.730,0.6990,0.909091,0.845859,0.0434,0.0401,0.000003,0.1290,0.624,0.483537
654,0.495,0.0495,0.909091,0.332949,0.0358,0.9840,0.937000,0.1020,0.223,0.112586
619,0.436,0.0356,0.000000,0.453370,0.0409,0.9920,0.961000,0.0895,0.238,0.525850


In [107]:
song_x_train, song_x_test, song_y_train, song_y_test = train_test_split(data, data_target, test_size= 0.2, random_state= 42)
df_result = []


model = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes= (16, 10,), activation= "logistic",max_iter= 1000, learning_rate_init= 0.01, alpha = 0.01), n_jobs = -1)
song_y_train = np.ravel(song_y_train)
song_y_test = np.ravel(song_y_test)
model.fit(song_x_train,song_y_train)
df_total_after["prediciton"] = model.predict(df_total_after.drop("tag", axis= 1))
tag = np.ravel(df_total_after[["tag"]].astype(int))
prediction = np.ravel(df_total_after[["prediciton"]].astype(int))
output = {"RMSE": np.sqrt(mean_squared_error(tag,prediction)), "accuracy_score":metrics.accuracy_score(tag,prediction)}
# df_result.append(output)
#         # print("RMSE", np.sqrt(mean_squared_error(song_y_test,song_y_predict)))
#         # print(metrics.accuracy_score(song_y_test, song_y_predict))
#         # print("------")
# df_output = pd.DataFrame(df_result)
# df_output
df_total_after
# song_x_test

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,tag,prediciton
0,0.746,0.754,0.090909,0.861097,0.0676,0.01520,0.000000,0.0486,0.746,0.647064,2,2
1,0.628,0.593,0.727273,0.899850,0.0323,0.09300,0.000002,0.0960,0.104,0.496042,2,3
2,0.620,0.805,0.454545,0.888045,0.0417,0.00564,0.000092,0.2850,0.586,0.478033,2,2
3,0.504,0.785,0.727273,0.905352,0.1040,0.07380,0.000000,0.2390,0.502,0.793408,2,2
4,0.560,0.872,0.272727,0.917886,0.1400,0.02390,0.000007,0.3710,0.682,0.502263,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
1496,0.688,0.662,0.545455,0.887292,0.0441,0.09710,0.000001,0.1720,0.549,0.435769,1,2
1497,0.521,0.877,0.636364,0.890255,0.0683,0.01320,0.080600,0.0929,0.199,0.830100,1,3
1498,0.287,0.473,0.454545,0.934769,0.0340,0.08580,0.000044,0.3830,0.577,0.868232,1,3
1499,0.673,0.704,0.545455,0.828834,0.3600,0.19600,0.000000,0.0898,0.372,0.264508,1,2


In [109]:
output

{'RMSE': 1.0086237085634806, 'accuracy_score': 0.6882078614257162}

In [117]:
df_difference = df_total_after[df_total_after.tag != df_total_after.prediciton]
df_difference[df_difference["tag"] == 2]

,Danceability,Energy,Key,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,tag,prediciton
1,0.628,0.593,0.727273,0.899850,0.0323,0.0930,0.000002,0.0960,0.104,0.496042,2,3
5,0.468,0.779,0.818182,0.904482,0.0779,0.1120,0.000373,0.1010,0.260,0.792381,2,3
6,0.527,0.835,0.545455,0.893689,0.0433,0.0166,0.000000,0.2490,0.654,0.501856,2,1
7,0.572,0.797,0.000000,0.919955,0.0497,0.0949,0.000000,0.2590,0.598,0.561260,2,1
9,0.461,0.834,0.181818,0.903800,0.0989,0.0232,0.000011,0.1400,0.471,0.556685,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
403,0.639,0.698,0.636364,0.854277,0.0702,0.7560,0.000000,0.1460,0.310,0.442823,2,3
405,0.802,0.848,1.000000,0.871091,0.0465,0.0118,0.395000,0.0567,0.165,0.465273,2,3
406,0.545,0.473,1.000000,0.824061,0.0312,0.2840,0.000000,0.0871,0.195,0.526263,2,3
409,0.696,0.734,0.636364,0.893783,0.0298,0.0485,0.000000,0.1830,0.711,0.392515,2,1


In [ ]:
# mean = df_output["accuracy_score"].mean()
# std = df_output["accuracy_score"].std()
# normal = np.random.normal(mean, std, size = len(df_output))
# plt.hist(normal, bins = 50, density= True)

In [ ]:
# mean = df_output["RMSE"].mean()
# std = df_output["RMSE"].std()
# normal = np.random.normal(mean, std, size = len(df_output))
# plt.hist(normal, bins = 30, density= True)

In [137]:
filename = "model.pkl"
with open (filename, "wb") as file:
    pickle.dump(model, file)

# load the saved model from file
filename = 'model.pkl'
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)


In [ ]:
test_energy = pd.read_csv("liked.csv")
test_energy = test_energy[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]]
# Normalize the DataFrame to be between 0 and 1
min_value = test_energy[["Tempo"]].min()
max_value = test_energy[["Tempo"]].max()
test_energy["Tempo"] = (test_energy[["Tempo"]] - min_value) / (max_value - min_value)

min_value = test_energy[["Loudness"]].min()
max_value = test_energy[["Loudness"]].max()
test_energy["Loudness"] = (test_energy[["Loudness"]] - min_value) / (max_value - min_value)

min_value = test_energy[["Key"]].min()
max_value = test_energy[["Key"]].max()
test_energy["Key"] = (test_energy[["Key"]] - min_value) / (max_value - min_value)
test_energy

In [210]:
test_energy

,Track URI,Track Name,Artist URI(s),Artist Name(s),Album URI,Album Name,Album Artist URI(s),Album Artist Name(s),Album Release Date,Album Image URL,Disc Number,Track Number,Track Duration (ms),Track Preview URL,Explicit,Popularity,ISRC,Added By,Added At
0,spotify:track:1Aq9S5teUzfZucr7rVRpSj,Oz.,spotify:artist:7kOrrFIBIBc8uCu2zbxbLv,yama,spotify:album:4R8cMdxOUDbHaHvwjEluEq,Versus the night,spotify:artist:7kOrrFIBIBc8uCu2zbxbLv,yama,2022-08-31,https://i.scdn.co/image/ab67616d0000b2730ceb00...,1,8,195866,https://p.scdn.co/mp3-preview/f42289bedfbe9b7b...,False,56,JPU902104200,NaN,2023-03-21T20:08:31Z
1,spotify:track:04TshWXkhV1qkqHzf31Hn6,Lemon,spotify:artist:1snhtMLeb2DYoMOcVbb8iB,米津玄師,spotify:album:052EiTRYh35MuDVJN9Emdh,STRAY SHEEP,spotify:artist:1snhtMLeb2DYoMOcVbb8iB,米津玄師,2020-08-05,https://i.scdn.co/image/ab67616d0000b273775e81...,1,8,255826,https://p.scdn.co/mp3-preview/5262679961609398...,False,69,JPU901800227,NaN,2023-03-20T05:49:04Z
2,spotify:track:2HovXsvcdJur52BOcYGydz,カタオモイ,spotify:artist:0bAsR2unSRpn6BQPEnNlZm,Aimer,spotify:album:336m0kejdM5Fkw2HUX46Bw,daydream,spotify:artist:0bAsR2unSRpn6BQPEnNlZm,Aimer,2016-09-21,https://i.scdn.co/image/ab67616d0000b273b56ada...,1,7,207360,https://p.scdn.co/mp3-preview/2d43a750be1ed140...,False,66,JPU901602293,NaN,2023-03-19T13:21:53Z
3,spotify:track:2wvMC5EyaaYQwBfiwwY2xE,Life's Been Good,spotify:artist:5bDxAyJiTYBat1YnFJhvEK,Joe Walsh,spotify:album:5yqBTSoJqE9EfApl2Pptva,"But Seriously, Folks...",spotify:artist:5bDxAyJiTYBat1YnFJhvEK,Joe Walsh,1978-06-01,https://i.scdn.co/image/ab67616d0000b273d6627c...,1,8,536226,https://p.scdn.co/mp3-preview/4220b4db522c3011...,False,63,USEE10901062,NaN,2023-03-18T17:45:34Z
4,spotify:track:2XHjFJVXYlEzoDvN82h8s5,Rocky Mountain Way,spotify:artist:5bDxAyJiTYBat1YnFJhvEK,Joe Walsh,spotify:album:6Sdkho6t0ure9XTzZXM2CK,"The Smoker You Drink, The Player You Get",spotify:artist:5bDxAyJiTYBat1YnFJhvEK,Joe Walsh,1973-06-18,https://i.scdn.co/image/ab67616d0000b2732df4d3...,1,1,315400,https://p.scdn.co/mp3-preview/0cad997484fdb994...,False,59,USMC17344516,NaN,2023-03-18T17:45:05Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634,spotify:track:4nVBt6MZDDP6tRVdQTgxJg,Story of My Life,spotify:artist:4AK6F7OLvEQ5QYCBNiQWHq,One Direction,spotify:album:7p1fX8aUySrBdx4WSYspOu,Midnight Memories (Deluxe),spotify:artist:4AK6F7OLvEQ5QYCBNiQWHq,One Direction,2013-11-25,https://i.scdn.co/image/ab67616d0000b2732f76b7...,1,2,245493,https://p.scdn.co/mp3-preview/9c3f89f926a70203...,False,81,GBHMU1300210,NaN,2014-11-09T03:06:32Z
1635,spotify:track:6BnONjR7itGMEqwxKTIlRM,Safe And Sound,spotify:artist:4gwpcMTbLWtBUlOijbVpuu,Capital Cities,spotify:album:5ps3FwS0qGdRCvXop8q9vn,In A Tidal Wave Of Mystery (Deluxe Edition),spotify:artist:4gwpcMTbLWtBUlOijbVpuu,Capital Cities,2013,https://i.scdn.co/image/ab67616d0000b2733a4040...,1,1,192789,NaN,False,0,USCA21203108,NaN,2014-11-09T03:05:50Z
1636,spotify:track:1eq1wUnLVLg4pdEfx9kajC,Rolling in the Deep,spotify:artist:4dpARuHxo51G3z768sgnrY,Adele,spotify:album:7n3QJc7TBOxXtlYh4Ssll8,21,spotify:artist:4dpARuHxo51G3z768sgnrY,Adele,2011-01-19,https://i.scdn.co/image/ab67616d0000b273ba7640...,1,1,228093,NaN,True,1,GBBKS1000335,NaN,2014-11-09T03:05:29Z
1637,spotify:track:6QPKYGnAW9QozVz2dSWqRg,Someone Like You,spotify:artist:4dpARuHxo51G3z768sgnrY,Adele,spotify:album:7n3QJc7TBOxXtlYh4Ssll8,21,spotify:artist:4dpARuHxo51G3z768sgnrY,Adele,2011-01-19,https://i.scdn.co/image/ab67616d0000b273ba7640...,1,11,285240,NaN,False,2,GBBKS1000351,NaN,2014-11-09T03:05:28Z


In [211]:

for i in range (len(pd.DataFrame(test_energy))) :
    song = pd.DataFrame(read["items"][i]).reset_index()
    song_uri = song.at[4, "track"]
    analysis = sp.audio_features(test_energy["Track URI"])
    analy = pd.DataFrame(analysis)
    output = pd.concat([output, analy])
output = output.drop(["mode", "duration_ms", "time_signature" , "type"], axis= 1)

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=1Aq9S5teUzfZucr7rVRpSj,04TshWXkhV1qkqHzf31Hn6,2HovXsvcdJur52BOcYGydz,2wvMC5EyaaYQwBfiwwY2xE,2XHjFJVXYlEzoDvN82h8s5,4ExVFhrJqFqgKSUAqDM5AZ,6KcCKEeszLQXeV0qSaUd1C,3jyhGjqueIqmJQR62nMFMR,73SpzrcaHk0RQPFP73vqVR,7zQs5RxMWQBVDTW9zl9CHu,5lPGfn9UM3HyLHZDs2Qups,3OdAoVRBtEg3PfsXsVoqFg,3o9kpgkIcffx0iSwxhuNI2,37mHnO9B6ZqcdLyuMMwZ8R,6QDyP4yKuqKfcK992yJxfB,4IouQaO9GkaHC7AtMErdSa,0XEEIG7IVzJUdkCg6taS9d,7jO0c4fG0NTFfj8SmPyNBC,0fs0LBPebyalY3KOh1aUWL,1Saksni0PtBv329gWAnU1F,0eFRk9wgFPxg6iXhDjvGAo,3yLdLoiGMwiM3GdY8w5WpP,4ESlJoWBExtFumtISwPabd,7uo2L5FAja3pWIBa3HJVeC,1TQXIltqoZ5XXyfCbAeSQQ,5lnnQwPdbF3uJioNX9Z5Gw,0z81zrkn4dqmRLJpRePjYc,4elhQCXOKEDhgFgE3Yxc8Q,19QNTNZtLdvUUTQi8redt8,1rJi64crBMOoS4g0YTInWo,5mu8z0sRt2GtImGpr2qgT6,0TRB7Y1CvHRoHg5bjSYINs,0umRfhrhFEz9jkEMTrUSkQ,5nQCxF5H1f0XcEWeaRVUED,2LawezPeJhN4AWuSB0GtAU,7aM0UMrjV7d6TWa8rdct5R,2RsO26SnRZg5btJ4FtZ8vD,5fJD7wh9yoSTpl9d5rgl7l,2zxJycVPzGvmHWmcvK5PWf,0muntSMuAZzcwYBBMZj6sf,4bHsxqR3GM

In [207]:
test_energy["prediciton"] = model.predict(test_energy)
len(test_energy)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- acousticness
- danceability
- energy
- instrumentalness
- key
- ...
Feature names seen at fit time, yet now missing:
- Acousticness
- Danceability
- Energy
- Instrumentalness
- Key
- ...


In [199]:
len(test_energy[test_energy['prediciton'] == 0])


12

In [200]:
len(test_energy[test_energy['prediciton'] == 1])


5

In [201]:
len(test_energy[test_energy['prediciton'] == 2])


24

In [202]:
len(test_energy[test_energy['prediciton'] == 3])

184